In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
### for reproducible experiments
from numpy.random import seed
seed(42)
from tensorflow.random import set_seed
set_seed(42)

/home/joshib/.cache/pypoetry/virtualenvs/dp-dpo-citation-bio-trainer-FaGiNfZ9-py3.7/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [4]:
import numpy as np
import pandas as pd
import git
import mlflow.keras
import os, sys
from gensim.models.wrappers import FastText

#import tensorflow_hub as hub
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
#import random
%matplotlib inline
pd.set_option('display.max_rows', None)
from tqdm import tqdm
sys.path.append('../citation_bio_trainer')
#from feature.SpacyFeaturizer import get_spacy_feats_from_text
from feature_window.Featurizer_window import Featurizer_window
from util.Utils import calulate_ser_jer, load_from_folder, pad_sequences, load_embedding_matrix, evaluate, log_mlflow_results
import warnings
from model.FTLSTM import calulate_ser_jer, get_model, plot_output
warnings.filterwarnings("ignore", category=DeprecationWarning) 

/home/joshib/.cache/pypoetry/virtualenvs/dp-dpo-citation-bio-trainer-FaGiNfZ9-py3.7/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


## LSTM with random embedding model

In [5]:
%%time
train_data_path = '/nlp/cs_mixed_20k/cs_mixed_20k_train/'
test_data_path  = '/nlp/cs_mixed_20k/cs_mixed_20k_test/'
eval_without_intra_newline_path  = '/nlp/eval_data_spacy_tokenized_extra_space_removed/'

sentences_train, sent_tags_train = load_from_folder(train_data_path)
sentences_test, sent_tags_test = load_from_folder(test_data_path)
sentences_eval1, sent_tags_eval1 = load_from_folder(eval_without_intra_newline_path)

CPU times: user 2min 39s, sys: 1.91 s, total: 2min 41s
Wall time: 2min 52s


In [6]:
xtrain, xvalid, ytrain, yvalid = train_test_split(sentences_train, sent_tags_train, test_size=0.1, random_state=42)

In [7]:
feat_config = {'max_vocab':100000,
               'lstm_feats':True, 
               'spacy_feats':True, 
               'google_feats': False, 
               'parscit_feats': False,
              'custom_feats': True, 
              'window':100,
              'step':90}

In [8]:
%%time
featurizer = Featurizer_window(feat_config)
print("loading train ...")
train_dict, tokenizer, maxlen = featurizer.fit_transform(xtrain, ytrain)
print("loading valid ...")
valid_dict = featurizer.transform(xvalid, yvalid)
print("loading test ...")
test_dict  = featurizer.transform(sentences_test, sent_tags_test)
print("loading evals ...")
eval1_dict  = featurizer.transform(sentences_eval1, sent_tags_eval1)

loading train ...


15817it [00:03, 4277.88it/s]
100%|██████████| 15817/15817 [00:00<00:00, 19576.20it/s]


total partitions = 167
loading valid ...


1758it [00:00, 13100.49it/s]
100%|██████████| 1758/1758 [00:00<00:00, 21199.68it/s]


total partitions = 18
loading test ...


1953it [00:00, 10020.15it/s]
100%|██████████| 1953/1953 [00:00<00:00, 20321.30it/s]


total partitions = 20
loading evals ...


1328it [00:00, 5180.01it/s]
100%|██████████| 1328/1328 [00:00<00:00, 10557.51it/s]


total partitions = 29
CPU times: user 3min 32s, sys: 10.5 s, total: 3min 42s
Wall time: 6min 56s


In [9]:
# %%time
# ### Load from file
# maxlen = 3861
# train_dict={}
# train_dict['labels'] = np.load('/nlp/temp/train_dict_labels.npy')
# train_dict['lstm_feats']=np.load('/nlp/temp/train_dict_lstm_feats.npy')
# train_dict['spacy_num_feats'] = np.load('/nlp/temp/train_dict_spacy_feats.npy')

# valid_dict={}
# valid_dict['labels']=np.load('/nlp/temp/valid_dict_labels.npy')
# valid_dict['lstm_feats']=np.load('/nlp/temp/valid_dict_lstm_feats.npy')
# valid_dict['spacy_num_feats']=np.load('/nlp/temp/valid_dict_spacy_feats.npy')

# test_dict={}
# test_dict['labels'] = np.load('/nlp/temp/test_dict_labels.npy')
# test_dict['lstm_feats']=np.load('/nlp/temp/test_dict_lstm_feats.npy')
# test_dict['spacy_num_feats']=np.load('/nlp/temp/test_dict_spacy_feats.npy')

# eval1_dict={}
# eval1_dict['labels'] = np.load('/nlp/temp/eval1_dict_labels.npy')
# eval1_dict['lstm_feats']=np.load('/nlp/temp/eval1_dict_lstm_feats.npy')
# eval1_dict['spacy_num_feats']=np.load('/nlp/temp/eval1_dict_spacy_feats.npy')

In [10]:
# def get_custom_feats(data_ls, maxlen):
#     feat_arr = np.zeros((len(data_ls), maxlen, 5), dtype='int8') #[[[0]* 5]* maxlen]* len(data_ls)
#     for ind in tqdm(range(len(data_ls))):
#         sent_ls = data_ls[ind].split(" ") 
#         for i in range(len(sent_ls) - 2):
#             if i>= maxlen:
#                 break
#             if i == 0:
#                 feat_arr[ind][i][0] = 1
#             elif sent_ls[i-1] == '\n':
#                 if sent_ls[i].isdigit() and len(sent_ls[i]) <= 2 and sent_ls[i+1] in ('.', ')'):
#                     feat_arr[ind][i][1] = 1
#                 elif sent_ls[i].isalpha() and len(sent_ls[i]) == 1 and sent_ls[i+1] in ('.', ')'):
#                     feat_arr[ind][i][2] = 1
#                 elif sent_ls[i] in ('[', '(') and (sent_ls[i+1].isdigit() and len(sent_ls[i+1]) <=2) and sent_ls[i+2] in (']', ')'):
#                     feat_arr[ind][i][3] = 1
#                 elif sent_ls[i] in ('[', '(') and (sent_ls[i+1].isalpha() and len(sent_ls[i+1]) ==2) and sent_ls[i+2] in (']', ')'):
#                     feat_arr[ind][i][4] = 1
#     return feat_arr#.reshape(len(data_ls), maxlen, 1)

In [11]:
# %%time
# train_custom_arr = get_custom_feats(xtrain, maxlen)
# valid_custom_arr = get_custom_feats(xvalid, maxlen)
# test_custom_arr = get_custom_feats(sentences_test, maxlen)
# eval1_custom_arr = get_custom_feats(sentences_eval1, maxlen)

### Neural parscit features (temporary fix)

In [77]:
%%time
from feature.ParsCitLSTM import ParsCitLSTM
c = {
        "model_file": "/nlp/parscit/parscit-29-latest.h5",
        "label_dict_file": "/nlp/parscit/labels.json",
        "tfhub_model_dir": "/nlp/parscit/resource/"}
model = ParsCitLSTM(model_config=c)

CPU times: user 23.3 s, sys: 1.52 s, total: 24.8 s
Wall time: 26.7 s


In [84]:
%%time
a,b,c = model.predict(sentences_train[0][0:200])

CPU times: user 112 ms, sys: 5.64 ms, total: 118 ms
Wall time: 99 ms


In [79]:
sentences_train[0][0:200]

'1 ) J. Chang et al . , “ 12.1 A 7 nm 256Mb SRAM in high - k metal - gate FinFET technology with write - assist circuitry for low - VMIN applications , ” 2017 IEEE International Solid - State Circuits '

In [13]:
# %%time
# train_df = pd.DataFrame([])
# valid_df = pd.DataFrame([])
# test_df = pd.DataFrame([])
#eval1_df = pd.DataFrame([])

# train_df['text'] = np.array(xtrain, dtype='object')
# valid_df['text'] = np.array(xvalid, dtype='object')
# test_df['text'] = np.array(sentences_test, dtype='object')
#eval1_df['text'] = np.array(sentences_eval1, dtype='object')

In [14]:
#%%time
# train_parscit = model.get_parscit_blocks(train_df)
# valid_parscit = model.get_parscit_blocks(valid_df)
# test_parscit  = model.get_parscit_blocks(test_df)
#eval1_parscit = model.get_parscit_blocks(eval1_df, chunk_size=40)


# train_parscit = pd.read_pickle('/nlp/temp/train_parscit.pickle')
# valid_parscit = pd.read_pickle('/nlp/temp/valid_parscit.pickle')
# test_parscit = pd.read_pickle('/nlp/temp/test_parscit.pickle')
# eval1_parscit = pd.read_pickle('/nlp/temp/eval1_parscit.pickle')

In [15]:
# def pad_dummy_feats(df, maxlen):
#     parscit = df.copy()
#     parscit_feats = list(parscit['parscit_feat']) 
#     parscit_padded = []
#     for ind in range(len(parscit_feats)):
#         parscit_mask = np.zeros((maxlen, 14), dtype='int8')
#         if len(parscit_feats[ind]) <= maxlen:
#             parscit_mask[0:len(parscit_feats[ind]), :] = parscit_feats[ind][:]
#         else:
#             parscit_mask[:] = parscit_feats[ind][0:maxlen,:]
#         parscit_padded.append(parscit_mask)
#     parscit_arr = np.array([i.tolist() for i in parscit_padded])
#     return parscit_arr

In [16]:
# %%time
# train_parscit_arr = pad_dummy_feats(train_parscit, maxlen)
# valid_parscit_arr = pad_dummy_feats(valid_parscit, maxlen)
# test_parscit_arr  = pad_dummy_feats(test_parscit, maxlen)
#eval1_parscit_arr = pad_dummy_feats(eval1_parscit, maxlen)

In [17]:
# %%time
# ## load saved files
# train_parscit_arr = np.load('/nlp/temp/train_parscit_arr.npy')
# valid_parscit_arr = np.load('/nlp/temp/valid_parscit_arr.npy')
# test_parscit_arr  = np.load('/nlp/temp/test_parscit_arr.npy')
# eval1_parscit_arr = np.load('/nlp/temp/eval1_parscit_arr.npy')

## Keras with pre-trained fast text embedding

In [18]:
#maxlen=100

In [19]:
%%time
#wiki_model = FastText.load_fasttext_format('/nlp/cc.en.300.bin')
#embedding_matrix = load_embedding_matrix(wiki_model, feat_config['max_vocab'], tokenizer.word_index, 300)
embedding_matrix=np.load('/nlp/temp/embedding_matrix.npy')

CPU times: user 81.5 ms, sys: 55.2 ms, total: 137 ms
Wall time: 999 ms


In [25]:
model_config = {'embedding': {'use':True, 'matrix':embedding_matrix, 'trainable':True},## trainableTrue is significantly better 
                'lstm': {'use':True, 'num': 1, 'units':50, 'dropout':0.2},
                'dense':{'use':False, 'num': 1, 'units':32, 'activation': 'relu', 'dropout':0.2},
                'optimizer': Adam(
        learning_rate=0.001,
        beta_1=0.0,
        beta_2=0.0,
        epsilon=1e-05,
        amsgrad=False,
    ),
                'output_activation' : 'sigmoid', 
                'batch_size': 32, ## lower the better
                'aux_feats': {'use':True, 'dim':17, 'place':'before_lstm'}, 
                'timedistributed':{'use':False}, ## no difference at all True or False
                'shuffle': True ## not much different than False
                
               }

In [20]:
%%time
model = get_model(feat_config, model_config, maxlen)
print(model.summary())

first line
after imput
after embedding
before bidirectional
after bidirectional
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     30000000    input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 100, 17)]    0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 100, 317)     0           embedding[0][0]                  
              

In [20]:
%%time
if feat_config['spacy_feats'] and not feat_config['parscit_feats'] and not feat_config['custom_feats']:
    print('only spacy not parscit not custom')
    train_aux = train_dict['spacy_num_feats']
    valid_aux = valid_dict['spacy_num_feats']
    test_aux  = test_dict['spacy_num_feats']
    eval1_aux  = eval1_dict['spacy_num_feats']
    
elif feat_config['spacy_feats'] and  feat_config['parscit_feats'] and not feat_config['custom_feats']:
    print("both spacy and parscit not custom")
    train_aux = np.concatenate((train_dict['spacy_num_feats'], train_parscit_arr), axis=-1)
    valid_aux = np.concatenate((valid_dict['spacy_num_feats'], valid_parscit_arr), axis=-1)
    test_aux  = np.concatenate((test_dict['spacy_num_feats'], test_parscit_arr), axis=-1)
    eval1_aux  = np.concatenate((eval1_dict['spacy_num_feats'], eval1_parscit_arr), axis=-1)
    
elif feat_config['spacy_feats'] and  feat_config['custom_feats'] and not feat_config['parscit_feats']:
    print("both spacy and custom not parscit")
    train_aux = np.concatenate((train_dict['spacy_num_feats'], train_dict['custom_feats']), axis=-1)
    valid_aux = np.concatenate((valid_dict['spacy_num_feats'], valid_dict['custom_feats']), axis=-1)
    test_aux  = np.concatenate((test_dict['spacy_num_feats'], test_dict['custom_feats']), axis=-1)
    eval1_aux  = np.concatenate((eval1_dict['spacy_num_feats'], eval1_dict['custom_feats']), axis=-1)
    
elif feat_config['spacy_feats'] and  feat_config['parscit_feats'] and feat_config['custom_feats']:
    print("all spacy and parscit and custom features")
    train_aux = np.concatenate((train_dict['spacy_num_feats'], train_parscit_arr, train_custom_arr), axis=-1)
    valid_aux = np.concatenate((valid_dict['spacy_num_feats'], valid_parscit_arr, valid_custom_arr), axis=-1)
    test_aux  = np.concatenate((test_dict['spacy_num_feats'], test_parscit_arr, test_custom_arr), axis=-1)
    eval1_aux  = np.concatenate((eval1_dict['spacy_num_feats'], eval1_parscit_arr, eval1_custom_arr), axis=-1)
else:
    print('only lstm features')

both spacy and custom not parscit
CPU times: user 1.06 s, sys: 691 ms, total: 1.75 s
Wall time: 1.74 s


In [ ]:
%%time
early_stop= EarlyStopping(monitor='val_loss',patience=3,verbose=0,mode='min',restore_best_weights=False, min_delta=0.0001)

## multi input
if model_config['aux_feats']['use']:
    print("multi input model")
    history = model.fit([train_dict['lstm_feats'], train_aux], train_dict['labels'], verbose=1, epochs=100, batch_size= model_config['batch_size'], \
                    validation_data=([valid_dict['lstm_feats'], valid_aux], valid_dict['labels']), callbacks=[early_stop], shuffle=model_config['shuffle'])
else:
    print("single input model")
    history = model.fit(train_dict['lstm_feats'], train_dict['labels'], verbose=1, epochs=100, batch_size= model_config['batch_size'], \
                        validation_data=(valid_dict['lstm_feats'], valid_dict['labels']), callbacks=[early_stop], shuffle=model_config['shuffle'])

plot_output(history)

multi input model
Epoch 1/100
5246/5246 [==============================] - 1299s 248ms/step - loss: 0.0041 - accuracy: 0.9987 - val_loss: 0.0010 - val_accuracy: 0.9996
Epoch 2/100
 540/5246 [==>...........................] - ETA: 18:51 - loss: 0.0011 - accuracy: 0.9996

In [24]:
print('done')

done


In [ ]:
#ls = [len(sent.split(" ")) for sent in sentences_test]

In [ ]:
#sentences_test[0]

### Analysis on validation data

In [25]:
%%time
#keras_model = mlflow.keras.load_model("s3://caps-s3-mlflow/artifacts/4/59e06c5ff45944f5bf56745b2e19cb05/artifacts/models")
if model_config['aux_feats']['use']:
    valid_probs = model.predict([valid_dict['lstm_feats'], valid_aux])
else:
    valid_probs = model.predict(valid_dict['lstm_feats'])

valid_probs = valid_probs.reshape(valid_probs.shape[0], valid_probs.shape[1])
valid_preds = np.where(valid_probs > 0.5, 1, 0)

valid_true_ls = [i[0:len(j)].tolist() for i,j in zip(valid_dict['labels'], valid_dict['tags_window'])]
valid_pred_ls = [i[0:len(j)].tolist() for i,j in zip(valid_preds, valid_dict['tags_window'])]
result_valid = evaluate(valid_true_ls, valid_pred_ls)

CPU times: user 30.3 s, sys: 824 ms, total: 31.2 s
Wall time: 30.2 s


In [26]:
result_valid

{'count': 18654,
 'count_citations': 45668,
 'mean_ser': 0.035972241367866975,
 'mean_jer': 0.03794619988733852,
 'mean_acc': 0.9998070034769974,
 'num_mistakes_seq': 274,
 'num_mistakes_all': 299,
 'mistakes_per_seq': 1.0912408759124088,
 'perc_mistakes_seq': 1.4688538651227618,
 'perc_mistake_per_citation': 0.6547254094770956}

In [31]:
result_valid

{'count': 18654,
 'count_citations': 45668,
 'mean_ser': 0.0392031376471711,
 'mean_jer': 0.038738042481207484,
 'mean_acc': 0.9996994326110984,
 'num_mistakes_seq': 441,
 'num_mistakes_all': 486,
 'mistakes_per_seq': 1.1020408163265305,
 'perc_mistakes_seq': 2.3641042135734964,
 'perc_mistake_per_citation': 1.0642025050363493}

In [95]:
result_valid

{'count': 18661,
 'count_citations': 45661,
 'mean_ser': 0.03800848197048829,
 'mean_jer': 0.03974595178978653,
 'mean_acc': 0.9997110495444855,
 'num_mistakes_seq': 428,
 'num_mistakes_all': 475,
 'mistakes_per_seq': 1.1098130841121496,
 'perc_mistakes_seq': 2.293553400139328,
 'perc_mistake_per_citation': 1.040275070629202}

In [55]:
result_valid

{'count': 18656,
 'count_citations': 45547,
 'mean_acc': 0.9996739582395737,
 'num_mistakes_seq': 498,
 'num_mistakes_all': 521,
 'mistakes_per_seq': 1.0461847389558232,
 'perc_mistakes_seq': 2.6693825042881647,
 'mistake_per_citation': 1.1438733615825412}

In [101]:
valid_preds_win = []
for ind in range(len(valid_preds)):
    valid_preds_win.append(valid_preds[ind][0:len(xvalid[ind].split(" "))])

In [ ]:
#keras_model = mlflow.keras.load_model("s3://caps-s3-mlflow/artifacts/4/977e2cc7b36040f79dde3d6303d26952/artifacts/models")
count_valid = 0
#valid_df = pd.DataFrame([])
valid_ls = []
for ind in range(len(valid_preds)):
    temp_ls = []
    pred = valid_preds[ind]
    true = valid_dict['labels'][ind]
    if (true == pred).all():
        pass
    else:
        count_valid += 1
        fp_ind = np.where((pred == 1) & (true == 0))[0]
        fn_ind = np.where((pred == 0) & (true == 1))[0]
        if len(fp_ind) > 0:
            for x in fp_ind:
                valid_ls.append([ind, 'FP'] + np.array(xvalid[ind].split(" "))[max(0, x-3):x+4].tolist())
                #print(ind, 'FP', np.array(xvalid[ind].split(" "))[max(0, x-3):x+4])
        if len(fn_ind) > 0:
            #print(ind, 'False negatives:', fn_ind)
            for x in fn_ind:
                valid_ls.append([ind, 'FN'] + np.array(xvalid[ind].split(" "))[max(0, x-3):x+4].tolist())
                #print(ind, 'FN', np.array(xvalid[ind].split(" "))[max(0, x-3):x+4])

In [ ]:
valid_mistakes_df = pd.DataFrame(valid_ls, columns=['index', 'error_type', 'x-3', 'x-2', 'x-1', 'x', 'x+1', 'x+2', 'x+3'])

### Analysis of test result

In [26]:
%%time
##### CHANGE THE KERAS MODEL LOADING PART ******************************************
model = mlflow.keras.load_model("s3://caps-s3-mlflow/artifacts/4/aea7f13e822d408e827726c2b972b93c/artifacts/models/")
if model_config['aux_feats']['use']:
    test_probs = model.predict([test_dict['lstm_feats'], test_aux])
else:
    test_probs = model.predict(test_dict['lstm_feats'])
test_probs = test_probs.reshape(test_probs.shape[0], test_probs.shape[1])
test_preds = np.where(test_probs > 0.5, 1, 0)
# test_true_ls = [i[0:len(j.split(" "))].tolist() for i,j in zip(test_dict['labels'], sent_tags_test_win)]
# test_pred_ls = [i[0:len(j.split(" "))].tolist() for i,j in zip(test_preds, sent_tags_test_win)]
# result_test = evaluate(test_true_ls, test_pred_ls)

CPU times: user 11.3 s, sys: 2.59 s, total: 13.8 s
Wall time: 21.2 s


In [27]:
%%time
take_mean = True
test_probs_win = []
for ind in range(len(test_probs)):
    #print(ind)
    test_probs_win.append(test_probs[ind][0:len(test_dict['sentences_window'][ind])])

test_probs_merged = []
window = 100
step = 90
gind = 0
for bigind in range(len(sentences_test)):
    temp = []
    seqlen =  len(sentences_test[bigind].split(" "))
    iter = 0
    #print(seqlen)
    #print(bigind)
    for smallind in range(gind, len(test_probs_win)):
        if len(temp) < seqlen:
            if iter ==0:
                temp+=test_probs_win[gind].tolist()
                iter +=1
                gind += 1
            else:
                if take_mean:
                    temp[-(window-step):] = np.mean([np.array(temp[-(window-step):]), test_probs_win[gind][0:(window-step)]], axis=0).tolist()
                    temp += test_probs_win[gind][(window-step):].tolist()
                    gind += 1
                else:
                    temp+=test_probs_win[gind][(window-step):].tolist()
                    gind += 1
        else:
            break
    test_probs_merged.append(temp)

tag2index = {'B-CIT':1, 'I-CIT':0}
test_preds_merged = [[int(t>0.5) for t in test_probs_merged[ind]] for ind in range(len(test_probs_merged))]
test_true_merged = [[tag2index[t] for t in sent_tags_test[ind].split(" ")] for ind in range(len(sent_tags_test))]

CPU times: user 1.43 s, sys: 63.4 ms, total: 1.49 s
Wall time: 1.42 s


In [28]:
%%time
result_test = evaluate(test_true_merged, test_preds_merged)

CPU times: user 7.82 s, sys: 0 ns, total: 7.82 s
Wall time: 7.81 s


In [29]:
result_test

{'count': 1953,
 'count_citations': 46092,
 'mean_ser': 0.0007581633612078364,
 'mean_jer': 0.002062706327176513,
 'mean_acc': 0.999909105171757,
 'num_mistakes_seq': 93,
 'num_mistakes_all': 121,
 'mistakes_per_seq': 1.3010752688172043,
 'perc_mistakes_seq': 4.761904761904762,
 'perc_mistake_per_citation': 0.26251844137811337}

In [30]:
result_test

{'count': 1953,
 'count_citations': 46092,
 'mean_ser': 0.0007581633612078364,
 'mean_jer': 0.002062706327176513,
 'mean_acc': 0.999909105171757,
 'num_mistakes_seq': 93,
 'num_mistakes_all': 121,
 'mistakes_per_seq': 1.3010752688172043,
 'perc_mistakes_seq': 4.761904761904762,
 'perc_mistake_per_citation': 0.26251844137811337}

In [35]:
result_test

{'count': 1953,
 'count_citations': 46092,
 'mean_ser': 0.0010308242989476224,
 'mean_jer': 0.004847970409573656,
 'mean_acc': 0.999821175050163,
 'num_mistakes_seq': 176,
 'num_mistakes_all': 225,
 'mistakes_per_seq': 1.2784090909090908,
 'perc_mistakes_seq': 9.011776753712237,
 'perc_mistake_per_citation': 0.4881541265295496}

In [101]:
result_test

{'count': 1953,
 'count_citations': 46096,
 'mean_ser': 0.0011575594383382826,
 'mean_jer': 0.004807974741352244,
 'mean_acc': 0.9998168478787155,
 'num_mistakes_seq': 181,
 'num_mistakes_all': 234,
 'mistakes_per_seq': 1.292817679558011,
 'perc_mistakes_seq': 9.267793138760881,
 'perc_mistake_per_citation': 0.5076362374175634}

In [31]:
%%time
tags = {'dataset':'cs_mixed_20k', 'data_split':'test', 'version':2} 
del model_config['embedding']['matrix'] 
opt = model_config['optimizer']
model_config['optimizer'] = str(opt.get_config())
log_mlflow_results(model, result_test, feat_config, model_config, tags)

2020/09/06 16:19:16 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under s3://caps-s3-mlflow/artifacts/4/7d154794377c4317b2c13fe3fb9aaaeb/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the  tracking store. If logging to a mlflow server via REST, consider  upgrading the server version to MLflow 1.7.0 or above.


CPU times: user 5.48 s, sys: 2.1 s, total: 7.58 s
Wall time: 12.1 s


In [30]:
count_test = 0
test_ls = []
for ind in range(len(test_preds_merged)):
    temp_ls = []
    pred = np.array(test_preds_merged[ind])
    true = np.array(test_true_merged[ind])
    if (true == pred).all():
        pass
    else:
        count_test += 1
        fp_ind = np.where((pred == 1) & (true == 0))[0]
        fn_ind = np.where((pred == 0) & (true == 1))[0]
        if len(fp_ind) > 0:
            #print(ind, 'fp', fp_ind)
            for x in fp_ind:
                ls = ['None']*7
                if max(0, x-3)==0:
                    ls[3-x:] = np.array(sentences_test[ind].split(" "))[max(0, x-3):x+4].tolist()
                else:
                    ls = np.array(sentences_test[ind].split(" "))[max(0, x-3):x+4].tolist()
                test_ls.append([ind, x, 'FP'] + ls)
                #print(ind, 'FP', np.array(sentences_test[ind].split(" "))[max(0, x-3):x+4])
        if len(fn_ind) > 0:
            #print(ind, 'False negatives:', fn_ind)
            #print(ind, 'fn', fn_ind)
            for x in fn_ind:
                ls = ['None']*7
                if max(0, x-3)==0:
                    ls[3-x:] = np.array(sentences_test[ind].split(" "))[max(0, x-3):x+4].tolist()
                else:
                    ls = np.array(sentences_test[ind].split(" "))[max(0, x-3):x+4].tolist()
                test_ls.append([ind, x, 'FN'] + ls)
                #print(ind, 'FN', np.array(sentences_test[ind].split(" "))[max(0, x-3):x+4])

In [31]:
test_mistakes_df = pd.DataFrame(test_ls, columns=['seq_index', 'token_index', 'error_type', 'x-3', 'x-2', 'x-1', 'x', 'x+1', 'x+2', 'x+3'])

In [32]:
test_mistakes_df = test_mistakes_df.replace('\n', 'newline')

In [36]:
test_mistakes_df[test_mistakes_df.error_type == 'FN']

,seq_index,token_index,error_type,x-3,x-2,x-1,x,x+1,x+2,x+3
0,3,1505,FN,York,.,newline,.,http://www.weizmann.ac.il,/,mcb
1,3,1525,FN,science-2002,.,newline,http://toreopsahl.com/datasets/.,newline,P.,Moscato
2,5,2904,FN,.,https://doi.org/10.1016/j.aca.2012.05.016,newline,(,2016,),.
3,16,87,FN,Energy,Agency,newline,4,.,newline,J.
4,31,254,FN,",",Tex,newline,Huang,",",A.-B.,&
6,54,32,FN,pp,203,newline,2,newline,.,National
9,117,221,FN,",",online,newline,Gladstone,Regional,Council,(
10,117,245,FN,",",online,newline,Gladstone,Regional,Council,(
11,117,263,FN,",",online,newline,Intergovernmental,Panel,on,Climate
12,117,307,FN,",",online,newline,North,Queensland,Conservation,Council


In [35]:
test_mistakes_df.error_type.value_counts()

FN    86
FP    35
Name: error_type, dtype: int64

In [58]:
test_mistakes_df.to_csv('../test_mistakes_windownew.csv')

In [ ]:
#test_mistakes_df[test_mistakes_df.error_type == 'FP']

### Analysis of eval dataset

In [ ]:
%%time
#keras_model = mlflow.keras.load_model("s3://caps-s3-mlflow/artifacts/4/1d953e89279b49fdb63cd7b3cb6c8b0b/artifacts/models")

In [37]:
model = mlflow.keras.load_model("s3://caps-s3-mlflow/artifacts/4/aea7f13e822d408e827726c2b972b93c/artifacts/models/")
if model_config['aux_feats']['use']:
    print('auxiliary features ...')
    eval1_probs = model.predict([eval1_dict['lstm_feats'], eval1_aux])
else:
    print('only lstm features ...')
    eval1_probs = model.predict(eval1_dict['lstm_feats'])
eval1_probs = eval1_probs.reshape(eval1_probs.shape[0], eval1_probs.shape[1])
eval1_preds = np.where(eval1_probs > 0.5, 1, 0)

auxiliary features ...


In [38]:
%%time
take_mean = True
eval1_probs_win = []
for ind in range(len(eval1_probs)):
    #print(ind)
    eval1_probs_win.append(eval1_probs[ind][0:len(eval1_dict['sentences_window'][ind])])

eval1_probs_merged = []
window = 100
step = 90
gind = 0
for bigind in range(len(sentences_eval1)):
    temp = []
    seqlen =  len(sentences_eval1[bigind].split(" "))
    iter = 0
    #print(seqlen)
    #print(bigind)
    for smallind in range(gind, len(eval1_probs_win)):
        if len(temp) < seqlen:
            if iter ==0:
                temp+=eval1_probs_win[gind].tolist()
                iter +=1
                gind += 1
            else:
                if take_mean:
                    temp[-(window-step):] = np.mean([np.array(temp[-(window-step):]), eval1_probs_win[gind][0:(window-step)]], axis=0).tolist()
                    temp += eval1_probs_win[gind][(window-step):].tolist()
                    gind += 1
                else:
                    temp+=eval1_probs_win[gind][(window-step):].tolist()
                    gind += 1
        else:
            break
    eval1_probs_merged.append(temp)

tag2index = {'B-CIT':1, 'I-CIT':0}
eval1_preds_merged = [[int(t>0.5) for t in eval1_probs_merged[ind]] for ind in range(len(eval1_probs_merged))]
eval1_true_merged = [[tag2index[t] for t in sent_tags_eval1[ind].split(" ")] for ind in range(len(sent_tags_eval1))]

CPU times: user 1.83 s, sys: 104 ms, total: 1.94 s
Wall time: 1.93 s


In [39]:
result_eval1 = evaluate(eval1_true_merged, eval1_preds_merged)

In [35]:
result_eval1

{'count': 1328,
 'count_citations': 56850,
 'mean_ser': 1.2982966348151225e-05,
 'mean_jer': 0.00041063361934943446,
 'mean_acc': 0.9999894054432367,
 'num_mistakes_seq': 18,
 'num_mistakes_all': 26,
 'mistakes_per_seq': 1.4444444444444444,
 'perc_mistakes_seq': 1.355421686746988,
 'perc_mistake_per_citation': 0.04573438874230431}

In [40]:
result_eval1

{'count': 1328,
 'count_citations': 56850,
 'mean_ser': 0.0007642510339866931,
 'mean_jer': 0.0075459564576609486,
 'mean_acc': 0.9998296369743739,
 'num_mistakes_seq': 353,
 'num_mistakes_all': 411,
 'mistakes_per_seq': 1.1643059490084986,
 'perc_mistakes_seq': 26.58132530120482,
 'perc_mistake_per_citation': 0.7229551451187335}

In [105]:
result_eval1

{'count': 1328,
 'count_citations': 56850,
 'mean_ser': 0.0007776320593378298,
 'mean_jer': 0.008467544104861686,
 'mean_acc': 0.9998095060436722,
 'num_mistakes_seq': 422,
 'num_mistakes_all': 463,
 'mistakes_per_seq': 1.0971563981042654,
 'perc_mistakes_seq': 31.77710843373494,
 'perc_mistake_per_citation': 0.8144239226033422}

In [30]:
result_eval1

{'count': 1328,
 'count_citations': 56850,
 'mean_ser': 0.0007637693631669535,
 'mean_jer': 0.006231187644864529,
 'mean_acc': 0.9998530866974515,
 'num_mistakes_seq': 300,
 'num_mistakes_all': 344,
 'mistakes_per_seq': 1.1466666666666667,
 'perc_mistakes_seq': 22.59036144578313,
 'mistake_per_citation': 0.6051011433597185}

### Visualize errors

In [41]:
count_eval1 = 0
eval1_ls = []
for ind in range(len(eval1_preds_merged)):
    temp_ls = []
    pred = np.array(eval1_preds_merged[ind])
    true = np.array(eval1_true_merged[ind])
    if (true == pred).all():
        pass
    else:
        count_eval1 += 1
        fp_ind = np.where((pred == 1) & (true == 0))[0]
        fn_ind = np.where((pred == 0) & (true == 1))[0]
        if len(fp_ind) > 0:
            #print(ind, 'fp', fp_ind)
            for x in fp_ind:
                ls = ['None']*7
                if max(0, x-3)==0:
                    ls[3-x:] = np.array(sentences_eval1[ind].split(" "))[max(0, x-3):x+4].tolist()
                else:
                    ls = np.array(sentences_eval1[ind].split(" "))[max(0, x-3):x+4].tolist()
                eval1_ls.append([ind, x, 'FP'] + ls)
                #print(ind, 'FP', np.array(sentences_eval1[ind].split(" "))[max(0, x-3):x+4])
        if len(fn_ind) > 0:
            #print(ind, 'False negatives:', fn_ind)
            #print(ind, 'fn', fn_ind)
            for x in fn_ind:
                ls = ['None']*7
                if max(0, x-3)==0:
                    ls[3-x:] = np.array(sentences_eval1[ind].split(" "))[max(0, x-3):x+4].tolist()
                else:
                    ls = np.array(sentences_eval1[ind].split(" "))[max(0, x-3):x+4].tolist()
                eval1_ls.append([ind, x, 'FN'] + ls)
                #print(ind, 'FN', np.array(sentences_eval1[ind].split(" "))[max(0, x-3):x+4])

In [42]:
#eval1_ls

In [43]:
eval_mistakes_df = pd.DataFrame(eval1_ls, columns=['seq_index', 'token_index', 'error_type', 'x-3', 'x-2', 'x-1', 'x', 'x+1', 'x+2', 'x+3'])

In [44]:
eval_mistakes_df = eval_mistakes_df.replace('\n', 'newline')

In [71]:
eval_mistakes_df

,seq_index,token_index,error_type,x-3,x-2,x-1,x,x+1,x+2,x+3
0,171,420,FP,",",Ekpa,newline,",",Cooper,TK,","
1,210,772,FN,),.,newline,18,Hunger,-,Craig
2,371,1286,FN,34,.,newline,.,A,Consensus,on
3,412,251,FN,76,.,newline,[,8]M.,Preene,","
4,412,561,FN,4260,.,newline,[,17]V.Hamm,",",B.B.Sabet
5,706,828,FN,after,subacute,newline,Monica,",",F.Z.,","
6,712,2833,FN,346−352,.,newline,66,.,http://www.ozm.cz/en/sensitivity-tests/small-s...,newline
7,776,282,FN,257,.,newline,[,10]J.W.Xi,",",Control
8,776,416,FN,129,.,newline,[,14]X.X.Zhao,",",L.J.Li
9,841,1458,FN,remodeling,in,newline,Saccharomyces,cerevisiae,mutants,affected


In [57]:
#sentences_eval1[841]

In [74]:
sentences_eval1[412]

'[ 1 ] H. Andrew , A.S. John , S. Helen , Geothermal energy recovery from underground mines , Renew . Sustain . Energy Rev. 15 ( 2011 ) 916–924 . \n [ 2 ] A.M. Jessop , J.K. Macdonald , H. Spence . Clean energy from abandoned mines at Springhill , Nova Scotia , Energy Sources . 17(1995 ) 93 - 108 . \n [ 3 ] A.M.Jessop , M.M.Ghomishei , M.J.Drury . Geothermal energy in Canda . Geothermics . 20(1991)369 - 386 . \n [ 4 ] G.R. Watzlaf , T.E. Ackman . Underground mine water for heating and cooling using geothermal heat pump system . Mine Water Environ . 25(2006)1 - 14 . \n [ 5 ] S. A. Ghoreishi Madiseh , Mory M. Ghomshei , F.P. Hassani , et . al . Sustainable heat extraction from abandoned mine tunnels : A numerical model . J. Renew . Sustain . Ener . , 4(2012 ) 1 - 16 \n [ 6 ] J. Raymond , R. Therrien , F. Hassani . Overview of geothermal energy resources in Quebec ( Canada ) mining environments . 10th Int . ‘ Mine water association congress - mine water and the environment ’ , Karlsbad , 

In [ ]:
eval1_dict['lstm_feats'][180][1879]

In [ ]:
tokenizer.word_index['.']

In [ ]:
eval1_dict['spacy_num_feats'][180][1880]

In [ ]:
"IS_ALPHA",  "IS_ASCII", "IS_DIGIT",
            "IS_LOWER",  "IS_UPPER", "IS_TITLE", 
            "IS_PUNCT",  "IS_SPACE", "IS_STOP", 
            "LIKE_NUM",  "LIKE_URL", "LIKE_EMAIL", #"LENGTH",
        ]

In [ ]:
eval1_parscit_arr[180][1879]

In [389]:
# for ind in range(100):
#     print(sentences_train[ind].split(" ")[0])

In [388]:
eval1_dict['lstm_feats'][0]

array([   63,     3,     1,    78,     2, 11609,   171,     2,  2422,
         179,     2,   512,  3094,     2,     1,  3550,     3, 40789,
       11617,     1,    10,  2244, 12023,    12, 95607,    10,  3636,
           7,  4653,  1668,     7, 18394,    43,    10,  1626,  4392,
         292,  8221,     3, 34022,     3,     1,     7,   135,     3,
           4,    84,     3,     1,   201,     2,  1504,  2010,     2,
        7020,   185,     2,     1,   212,     2,  4171,    57,     2,
        1124,  5467,     2,     1,   253,     2, 14649,  5467,     3,
         379,    71,     8, 11617,  5507,     1,    38,   292,     7,
       11507, 20550,    43,  5540, 11221,  5456, 10545, 59296,     3,
       10865,     3,     1,     7,  1199,     3,     4,    87,     3,
           1])

In [390]:
tn, fp, fn, tp = confusion_matrix(y_true=eval1_true_merged[0], y_pred=eval1_preds_merged[0]).ravel()

In [56]:
eval_mistakes_df.to_csv('../eval_mistakes_windownew.csv', index=False)

### Save eval results to file

In [ ]:
ind = 0
eval_path_dict = {}
for fpath in os.listdir(eval_without_intra_newline_path):
    if fpath not in ['data-gen-config.json', 'data_generation_stats.csv'] and ".csv" in fpath:
        eval_path_dict[ind] = fpath
        ind += 1

In [ ]:
eval_folder_path = '/nlp/eval_data_retok_predictions_f3a9cc61e30f460499b95a4c2b7957ae'
if not os.path.exists(eval_folder_path):
    os.makedirs(eval_folder_path)
for ind in range(len(eval_preds1)):
    #print(ind)
    df = pd.DataFrame([], columns=['x', 'y'])
    seq_len = min(len(sentences_eval1[ind].split(" ")), maxlen)
    df['x'] = sentences_eval1[ind].split(" ")[0:seq_len]
    df['y'] = eval_preds1[ind][0:seq_len]
    df.to_csv(os.path.join(eval_folder_path, eval_path_dict[ind][0:-4] + '_pred.csv'))

### Log Eval results to MLFlow

In [36]:
tags = {'dataset':'cs_mixed_20k', 'data_split':'eval_with_intra_newline_path','version':2}
#del model_config['embedding']['matrix'] 
#opt = model_config['optimizer']
#model_config['optimizer'] = str(opt.get_config())
log_mlflow_results(model, result_eval1, feat_config, model_config, tags)

2020/09/06 16:20:20 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under s3://caps-s3-mlflow/artifacts/4/aea7f13e822d408e827726c2b972b93c/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the  tracking store. If logging to a mlflow server via REST, consider  upgrading the server version to MLflow 1.7.0 or above.


### Saving features to file

In [ ]:
# %%time
# np.save('/nlp/temp/train_dict_labels.npy', train_dict['labels'])
# np.save('/nlp/temp/train_dict_lstm_feats.npy', train_dict['lstm_feats'])
# np.save('/nlp/temp/train_dict_spacy_feats.npy', train_dict['spacy_num_feats'])

# np.save('/nlp/temp/valid_dict_labels.npy', valid_dict['labels'])
# np.save('/nlp/temp/valid_dict_lstm_feats.npy', valid_dict['lstm_feats'])
# np.save('/nlp/temp/valid_dict_spacy_feats.npy', valid_dict['spacy_num_feats'])

# np.save('/nlp/temp/test_dict_labels.npy', test_dict['labels'])
# np.save('/nlp/temp/test_dict_lstm_feats.npy', test_dict['lstm_feats'])
# np.save('/nlp/temp/test_dict_spacy_feats.npy', test_dict['spacy_num_feats'])

In [ ]:
# np.save('/nlp/temp/eval1_dict_labels.npy', eval1_dict['labels'])
# np.save('/nlp/temp/eval1_dict_lstm_feats.npy', eval1_dict['lstm_feats'])
# np.save('/nlp/temp/eval1_dict_spacy_feats.npy', eval1_dict['spacy_num_feats'])